# Lab 2: Methods in Linear Regression

In this lab we will canvas various techniques in linear regression analysis and apply them to the numerical variable in the Ames dataset. Some of the techniques, like Ridge Regression and Lasso Regression are already implemented in sci-kit learn, or statsmodels.api but some, like best subset regression, are not. 

In this lab, we'll be looking at __Linear Regression__, __Best Subset Selection__, __Forward Subset regression__, __Ridge Regression__ and __Lasso Regreesion__. We will be comparing these models using the number of degrees of freedom. We will be benchmarking these techniques using cross validation, that is by splitting our data into a training set and a test set and comparing the score on the test set. 

We will use two libraries in this lab: Sci-kit Learn and Statsmodel.api. Both have their uses and often implement similar functionality. The main difference is that Statsmodel.api is designed around statical computing, giving rich, human readable feedback the way the R might but at the cost that different functions behave differently. 

On the other hand, Sci-kit Learn is trying to implement a unified way of training and using any machine learning model. That is the syntax is the same for linear model, quadratic models, clusters, $k$-NN, etc. For example, for any Sci-kit model, `model.fit(X,Y)` will train the model on the data `X` with labels `Y` and `model.score(XT,YT)` will score the model on test data. 

We will discuss both libraries in this lab. 

#### Getting back to where we were

Lets quickly download the Ames dataset and throw out the outliers recomended in the documentation. Recall

<div class="alert alert-block alert-info">
SPECIAL NOTES:<br>
There are 5 observations that an instructor may wish to remove from the data set before giving it to students (a plot of SALE PRICE versus GR LIV AREA will indicate them quickly). Three of them are true outliers (Partial Sales that likely don’t represent actual market values) and two of them are simply unusual sales (very large houses priced relatively appropriately). I would recommend removing any houses with more than 4000 square feet from the data set (which eliminates these 5 unusual observations) before assigning it to students.<br><br>

http://jse.amstat.org/v19n3/decock/DataDocumentation.txt
</div>

Lets load in our standard libraries and dataset:

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

url = 'https://raw.githubusercontent.com/tipthederiver/Math-7243-2020/master/Datasets/Ames/train.csv'

ames = pd.read_csv(url, error_bad_lines=False)
ames

Our first job is to clean the outlier mentioned in the documentation for the Ames dataset. We will follow their suggestion and remove all houses over 4000 square feet of above ground living area.

In [ ]:
z = ames['GrLivArea']+ames['BsmtUnfSF']<4000
print("Number of records removed:",len(ames) - sum(z))
data = ames[z]

Indeed, we can check quickly and see that this removes our weird outliers from before.

In [ ]:
f, axes = plt.subplots(1,2)
f.set_size_inches(10,4)

axes[0].plot(ames['GrLivArea'],ames['SalePrice'],'.')
axes[0].plot(data['GrLivArea'],data['SalePrice'],'.')
axes[1].plot(data['GrLivArea'],data['SalePrice'],'.',color="C1")

axes[0].set_title("Excised Data")
axes[1].set_title("Cleaned Data")

#### Removing catagorical varaibles

The next step is to cut out all of the catagorical varaibles. There are ways to use regression to fit cataogrical variables but we will not be focusing on those methods in this lab. As before we will do this using the `DataFrame.include` function and only including the numerical data types `int64` and `float64`.

In [ ]:
data = data.select_dtypes(include=['int64','float64'])
display(data.head(5))

It is worth going through the documentation and checking if any of the variables that are represented numerically in the dataframe actually represent categorical variables (like error codes, phone number prefixes, etc).  In fact, the documentation tells us that __MSSubClass__ is a categorical variable, with each number representing a zoning code. We should drop it using `DataFrame.drop()`.

* `DataFrame.drop(columns=[name1,name2,...])` drops a list of column by name. It then returns the new dataframe which must be saved into a variable. `drop` can also be used to drop rows. 

In addition, __ID__ is just a label and could be dropped. It should be noted that the ML school of thought would say we should consider _not_ dropping these indices. The idea is very much "throw in all the data, maybe there's some correlation there we don't see." We will drop the data for this lab just for practice.  

In [ ]:
data = data.drop(columns='MSSubClass')
data.head(5)

## Cross validation

We want to measure how well a classifier will work on unseen data. To that end, we usually split our data into two pieces, a __training set__ that we will use to train the regression function, and a __test set__ that we will use to evaluate the function. This method sacrifices some of our data to (hopefully) increase our accuracy by giving  to get a better idea of how good our fit actually. If overfitting is a concern, it is vital to test a regression function against data that is was not trained on. Typically in Machine Learning, we will test many models with the train test split and use their relative accuracies to determine which model should be used for that particular problem. The model can then be trained against to entire dataset before being put into production.

We will use a 80%-20% train-test split. Since we have 1449 data points, the training set will be 1159 rows and the test set will be 290 rows. We can use Pandas built in `df.sample` function to sample the dataframe and the drop the training rows the form the test set.

* `DataFrame.sample(n=None, frac=None, replace=True, random_state=None)` Sample elements of a dataframe. If `n` is specified it returns a number of elements from the dataframe, if `frac` is specified it returns a fraction of the dataframe. The `replace` determines whether the sampling is done with or without replacement. It can be set to `False` but defaults to `True`. The `random_state` parameter is a random seed.  

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html

We then split the train and test into input and target data.

In [ ]:
print("Train Size:", 1449*.8)
print("Test Size:", 1449*.2)

Test_Size = 1159

train=data.sample(n=Test_Size,replace=False,random_state=150)
test=data.drop(train.index)

print("Train Shape", train.shape)
print("Test Shape", test.shape)

X_train = train.drop(columns=['SalePrice','Id'])
Y_train = train['SalePrice']

X_test = test.drop(columns=['SalePrice','Id'])
Y_test = test['SalePrice']

## Linear Regression

We now want to perform our first predictions using linear regression. We will do a quick one variable fit, only fitting the feature __1stFlrSF__ using both linear algebra and then using the sci-kit learn library and compare the fits. 

For a single feature, we want to fit a 2 vector of constants $[\beta_0,\beta_1]$ to 
$$
Y = \beta_0 + X\beta_1 = {X}^T\beta = [1,X] \left[ \begin{matrix} \beta_0\\\beta_1 \end{matrix} \right] \,.
$$
The solution will be given by
$$
\beta = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}\,,
$$
where $\mathbf{X}$ is the $1159 \times 2$ data matrix whose first column is filled with `1`'s and whose second column contains the data points $x_i$. Similarly, $\mathbf{y}$ is a column vector of target values.

We will convert the dataframes __X_train__ and __Y_train__ to `numpy` matrices __X__ and __y__ respectively to perform this calculation, but lets start with one variable. We will try to fit __1stFlrSF__ to __SalePrice __.

In [ ]:
X = np.matrix(X_train['1stFlrSF'])
Y = np.matrix(Y_train)

# Check the shape, they need to be column vectors.

print("X shape",X.shape)
print("Y shape",Y.shape)

We need to reshape these vectors into column vectors. When using reshape, you're always constrained by the fact that the new matrix shape needs to be compatible with the old shape. Practically, that means we can let Python figure out one dimension for us. You specify the dependent dimension by -1. For example,

`DataFrame.reshape(-1,1)` return a vector with 1 row and however many columns are required to match the original shape. 

In [ ]:
X = X.reshape(-1,1)
Y = Y.reshape(-1,1)

Y.shape

Now, we add a column of 1's for the affine term

In [ ]:
Xa = np.append(np.ones(X.shape),X,1)

And compute beta.

We now plot the data against the fit line.

Lets check how we did on our test set, both with a graph and numerically. Graphically, we can just plot the regression line again the test data scatter plot. We can evaluate the model numerically but computing the residual sum squared
$$
RSS(\beta) = \sum_{i=1}^N (y_i - x_i^T\beta)^2\,,
$$
the root mean square
$$
RMS(\beta) = \left(\frac{1}{N}(y_i - x_i^T\beta)^2\right)^{\frac12}\,,
$$
and the $r^2$ value
$$
r^2 = 1 - \frac{RSS(\beta)}{\sum_{i=1}^N (y_i - E[y])^2}\,.
$$

On the training data, the RMS is

Lets set up the matrices for the test data.

We can now compute the RSS and RMS on the test data:

In [ ]:
RSS = (YT - XTa*betas).T*(YT - XTa*betas)
RMS = np.sqrt(RSS/len(YT))
print("The Residual Sum Square is", RSS)
print("The Root Mean Square is", RMS)

Rsq = 1 - RSS/((YT - YT.mean()).T*(YT - YT.mean()))

print("R^2 Score is", Rsq)

In [ ]:
plt.plot(XT,YT,'o')
plt.plot(XT,XTa*betas)

#### Exercise: 

We could also consider other metrics, even when evaluating our result. For example, calculate the __average absolute difference__ between the predicted value and true value for both the test and train data.

#### How do we understand the results

Comparing the RMS values for the train and test data you may find that the results on the training data are better, or that the results on the test data are better. This means that our training data is reasonably representative of the whole set, but you should note that different choices of random splitting in the data will lead to different $\beta$ values and so different goodnesses of fit. This variability with respect to the sampling of the distribution is what is being captured in the variance. 


#### Exercise: 
Try running the code about for two different seeds in the training test split: `random_state=150` and `random_state=200`. Compare the results. How do you explain them?

## Linear Regression with Sklearn

We will now use sci-kit learn's built in regression function. We import `LinearRegression` from `sklearn.linear_model`, the toolkit of linear models in sci-kit learn. We then set up a linear regression object using

`lr = LinearRegression()`

An object like a linear regression object is a structure like a dataframe. It stores data, but also has a series of utility function associated with it. It is a self contained machine that we can put data into, turn a crank (by calling functions) and have it process and return that data. In this case, it takes in training data and fits a linear regressor to it. We can then ask it return the parameters of that regressor with 

* `lr.coef_` Returns the slope coefficients of regression $\beta_1,\ldots, \beta_p$.
* `lr.intercept_` Returns the intercept of regression $\beta_0$.
* `lr.score(X_data,Y_data)` Returns the $r^2$ score on `X_data` and `Y_data`.

Note, that `LinearRegression` wants the data in the form we've been using in class: the $X$ data should be a $N$ by $p$ matrix of data points and the $y$ data should be a $N$ by 1 column vector. 

In [ ]:
## Resample the data if needed

train=data.sample(n=Test_Size,replace=False,random_state=150)
test=data.drop(train.index)

X_train = train.drop(columns=['SalePrice','Id'])
Y_train = train['SalePrice']

X_test = test.drop(columns=['SalePrice','Id'])
Y_test = test['SalePrice']

X = np.matrix(X_train['1stFlrSF']).T
Y = np.matrix(Y_train).T

XT = np.matrix(X_test['1stFlrSF']).T
YT = np.matrix(Y_test).T

print('X shape:', X.shape,'Y shape', Y.shape)

Note in the above that since Sci-kit learn expects a $N\times p$ matrix for `X` and a column vector for `Y`, we will need to transpose them from their cut form. 

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X,Y)

print([lr.coef_,lr.intercept_])
print(lr.score(XT,YT))

## Regression with statsmodels.api

Although in this class we will be focusing on sci-kit learn, there's another important statistics library to be aware of, statsmodels.api. For regression in particular, statsmodels.api is just a much better tool, performing all of the statistical analysis you would like. The only caveat is that stats model doesn't naturally fit to a constant term, so we must again add a column of 1's. But statsmodels.api has a function to preform that task.

* `sm.OLS(Y_Data, X_data)` OLS returns an ordinary least squares object which can then be fit with `ols.fit()`. The fit object returned by `ols.fit()` has a robust `.summary()` method that gives all us rich and detailed information about the regression fit.

Note: in addition to fitting, OLS has functions to deal with missing values and built in ridge and lasso regression functions. 

In [ ]:
import statsmodels.api as sm

ols = sm.OLS(Y, Xa)
ols_result = ols.fit()

ols_result.summary()

Note the OLS has also computed the standard error, the $t/z$-statistic and the $95%$ confidence interval.

## Multilinear Regression: Cleaning Data

It actually often takes less computational time to compute a linear regression than it does to plot it. We want to fit the set of training data with all the numerical features to the sale price, but before we do we have to deal with those `NaN` values. For example, if we just try to compute the linear regression now it will return an error. 

Now, lets try to train against all of `X_train` and `Y_train` using sci-kit learn.

In [ ]:
lr = LinearRegression()
lr.fit(X_train,Y_train)

Fixing missing values is fiddly, and should be done with care. Whatever choice you make will effect your fit, and as a result will effect your accuracy. In general, data cleaning is the usually more than half the battle. 

To find the `NaN` values, we will use the `DataFrame.isnull().sum()` to return all of the null values, and them sum them along columns. 

We find that __LotFrontage__, __MasVnrArea__, and __GarageYrBlt__ are the only numerical features with null values. Lets take a look at each of these values individually. After we make a choice for each variable, we will use

* `DataFrame.fillna(Value, inplace=True)` fills all of the `NaN` values in an array with the contents of `Value`. 

__MasVnrArea__ is "Masonry veneer area in square feet". Dumping the data we find a lot of zero values, so its probably safe to assume that 8 values can be set to 0 or the median value. 

For __GarageYrBlt__, a value of `NaN` probably means there's no garage. Lets check just to make sure by looking at the __GarageArea__ variable for every `NaN` value.

Finally, for __LotFrontage__ we assume that almost every lot has some frontage road so we don't want to set this to 0. There are more sophisticated solutions, but the simplest is just to find the median value and set the null values to it. 

Finally, lets check to make sure we've elemented all missing values.

### Cross validation and Fitting
We now reestablish our train/test split

In [ ]:
Test_Size = 1000

train=data.sample(n=Test_Size,replace=False)
test=data.drop(train.index)

print("Train Shape", train.shape)
print("Test Shape", test.shape)

X_train = train.drop(columns=['SalePrice','Id'])
Y_train = train['SalePrice']

X_test = test.drop(columns=['SalePrice','Id'])
Y_test = test['SalePrice']

And train the data using statsmodels.api. Remember that we have to add a column to `X_train` to account for the constant term. Luckly, statsmodels has built in function for that `sm.add_constant(X_data)`

In [ ]:
ols = sm.OLS(Y_train, sm.add_constant(X_train))
ols_result = ols.fit()

ols_result.summary()

Now we want to score this model on test data. Unfortunently statsmodels.api doesn't have a built in test function, but we can use our own pretty easily. 

In [ ]:
def RSS(y,Y):
    y = np.matrix(y).reshape(-1,1)
    Y = np.matrix(Y).reshape(-1,1)
    
    return (y-Y).T*(y-Y)

def RMS(y,Y):    
    return np.sqrt(RSS(y,Y))/len(y)

def Rs(y,Y): 
    y = np.matrix(y).reshape(-1,1)
    Y = np.matrix(Y).reshape(-1,1)
    
    return 1 - RSS(y,Y)/((Y - np.mean(Y)).T*(Y - Y.mean()))

In [ ]:
pred = ols_result.predict(sm.add_constant(X_test))
Rs(pred, Y_test)
#RMS(pred, Y_test)

## Ridge Regression and Lasso Regression

Now that we have a baseline, we try out our new methods. We will look at ridge regression and lasso regression first, since they implemented in OLS. 

A `statsmodel.api.ols` object has a regularized regression fit function `ols.fit_regualrized()` that implements both lasso and ridge regression. In fact, it minimizes the loss function

$$
\textbf{Loss}(\beta) = \frac{1}{2n}\textbf{RSS} + \alpha\left(\frac{(1-\lambda)}{2}\sum_i \beta_i^2 + \lambda\sum_i|\beta_i|\right)
$$

In ridge regression, $\lambda = 0$ and so we are minimizing $RSS(\beta)$ subject to the additional constrain that $||\beta||^2$ is small. For a fixed value of the constraint $||\beta||^2 = R$, the $\beta$'s are constrained to a sphere. 

The addition of the lasso regression condition leads to a non-smoothness around the axes for the $\beta$ constraint. These two terms are both examples of __regulators__, and will be discussed more in class. 


This linear combination of loss functions is known as __elastic net regression__. For a little demo of the interpolation between the bounded regions for the variables take a look at this Desmos applet: The applet shows how the shape of the possible $\beta_i$ change for a fixed value of the regulator given different $\lambda$ and $\alpha$.  In the demo, $a$ is the $\alpha$ variable and $b$ is the $\lambda$ variable. 

https://www.desmos.com/calculator/a4a6qgitr7


The `ols.fit_regualrized()` takes the following form, where $\lambda$ is `L1_wt`:

* `OLS.fit_regularized(alpha=0.0, L1_wt=1.0, start_params=None)` Here, `alpha` is the overall weight $\alpha$ and `L1_wt` is the coefficient of the $L_1$ norm. 

## Ridge Regression

First, lets compute the ridge regression. Note that not all of the functionality has been added to statsmodels.api's OLS class yet. In particular, summary doesn't work. But we can still compute $r^2$ values on the test set using the functions we defined above. Pick a value at random, let's compute with 

In [ ]:
ols = sm.OLS(Y_train, sm.add_constant(X_train))
ols_result = ols.fit_regularized(L1_wt=0,alpha=1)

Rs(ols_result.predict(sm.add_constant(X_test)),Y_test)

We're probably already doing worse than expected. Lets predict for 50 $\alpha$ values from 0 to 1. We will save the $r^2$ value in in a list, by creating an empty list and appending each value into it. 

In [ ]:
print(np.max(r_sqds))
plt.plot(alphas, r_sqds)

If we add a few lines of code to save the $\hat \beta$ values at each iteration, you can plot them again eachother. Of course, the vast difference in $\hat \beta$ values we saw in the ordinary linear regression model indicates that they wont be very meaningful on the same graph (the largest are on the order of $10^6$, the smallest on the order of $10^{-1}$). 

The simplest way to overcome this, if we are just intersted in the dynmics of the varaibles, is to normalize the largest $|\beta_i^{\alpha}|$ to 1 for each feature. This can be done by dividing by the max of the absolute value. 

In [ ]:
f, axes = plt.subplots()
f.set_size_inches(14,8)

plt.plot(alphas,betas/betas.abs().max())
plt.legend(["Coeff"]+list(X_train))

## Lasso Regression

To compute the Lasso regression we can recycle our code from above, but now we set `L1_wt=1`, turning off the ridge parameters and turning of the Lasso. We will also have to modify our $alpha$ values since the $\beta$ are penalized much less for being large. 

In [ ]:
print(np.max(r_sqds))
print(np.max(rss_s))
plt.plot(alphas, r_sqds)

In [ ]:
f, axes = plt.subplots()
f.set_size_inches(14,8)

plt.plot(alphas,betas/betas.abs().max())
plt.legend(["Coeff"]+list(X_train))

# Subset Selection:

We will implement subset selection algorithms. Before we could lean on built in libraries, but we will now have to implement the algorithms ourself. Luckily, subset selection is a relatively straight forward idea. 

## Forward subset selection:

We will begin with forward subset selection using the $r^2$ value. The idea is to greedily add variables to our fitting, starting with the best single variable predictor and adding new variables one at a time. 

We will implement a function that returns the best predictor to add to a list of preselected predictors. This function will take the form

* `best_predict(inc, X_train, Y_train)` Return the single feature which improves the $r^2$ score the most. Here, `inc` is a list of predictors that we have already included.

Our `best_predict` function will be laid out as follows:

* First, get a list of columns in `X_train` that are no in already in `inc`.
* Set up lists to hold the column names, and the $r^2$ value of the model after adding each new column.
* Loop of the features found in the first step. For each feature `p`:
* * Fit a linear model to the features in `inc` and the feature `p`.
* * Score the model on the test data.
* * Append the feature name to the Columns list and append the computed $r^2$ value to the list of $r^2$ values. 
* After running the loop above, we get the index of the maximum $r^2$ value computed above. This will correspond to the list of features with the biggest improvement. 
* Return the maximum $r^2$ value, along with the list of features. 

We can then call this function iteratively, feeding the input of the previous run into the next run. 

In [ ]:
def best_predict(inc, X_train, Y_train, X_test,Y_test):
    rem_pred = [p for p in X_train.columns if p not in inc]
    
    RSquared = []
    Columns = []
    
    for p in rem_pred:
        olst = sm.OLS(Y_train, sm.add_constant(X_train[inc+[p]]))
        olst_result = olst.fit()    
        
        R_sq = Rs(olst_result.predict(sm.add_constant(X_test[inc+[p]])),Y_test)
        
        RSquared.append(R_sq)
        Columns.append(inc+[p])
        
    z = RSquared.index(max(RSquared))
    print("Maximum r^2", RSquared[z])
    print("Columns", Columns[z])
    
    return([float(RSquared[z]),Columns[z]])

In [ ]:
varlist = []
r_list = []

for i in range(0,20):
    r,varlist = best_predict(varlist,X_train, Y_train, X_test, Y_test)
    r_list = r_list + [r]
    
plt.plot(r_list)

## (Extra) Best subset selection and residual bootstrapping:

Best subset selection is impractical on large arrays of variables, but it's good to understand how it can be implemented. The code below uses itertools to iterate over all subsets of the range. Notice though that even for subsets of size 3 it really begins to chugg. This naive version of best subset selection shouldn't be used for data sets with this many features. 

In [ ]:
r_list

In [ ]:
import itertools
p = len(list(X_train)) # Number of featuers

names = list(X_train)

for d in range(1,3):

    subsets = list(itertools.combinations(range(0,p), d))
    print(p, "features,", len(subsets),"subsets of length ", d)

    fits = pd.DataFrame(columns=['cols','r sq train','r sq test','parameters'])

    for s in subsets:
        si = [names[i] for i in s]

        lr = LinearRegression()
        lr.fit(X_train[si],Y_train)

        Y_head_lr = lr.predict(X_test[si])
        rtr = Rs(Y_head_lr,Y_test)

        Y_head_lr_train = lr.predict(X_train[si])
        rte = Rs(Y_head_lr_train,Y_train)

        fits = fits.append({'cols':s,'r sq train':rtr,'r sq test':rte,'parameters':lr.coef_},ignore_index=True)
    
    best = fits.sort_values("r sq test",ascending=False).head(1)
    print(best[['cols','r sq train','r sq test']])
    print(best['parameters'])
    print()

## Exercise:

Find the best single varaible to drop from the full linear linear fit. That is, do one step of backwards subset selection. 

## Exercise:

Keeping in mind that the training/test split was arbitrary, what do you expecct the best method for prediction on the Ames data set to be? Justify your answer. 

## Exercise:

The `ols.fit_regularized(L1_wt=1,alpha=alp)` function alloes us to use eleastic net regression allow us to mix the ridge and lasso methods.

Splitting the data using 

`train=data.sample(n=Test_Size,replace=False, random_state=200)`

where the `random state` seed has been set to 200 will help normalized the results. What is the best pair $(\alpha,\lambda)$ to fit with?

## (Extra) Degrees of Freedom

As we construct our various linear models, we want to compare them based on their degrees of freedom. Recall that the number of degrees of freedom of a model $\hat h:\mathcal{X}\to \mathcal{Y}$ with $\hat y_i = h(\hat x_i)$ is 
$$
df(\hat f) = \frac{1}{\sigma^2} \sum_{i=1}^N \text{Cov}(\hat y_i,y_i) = \frac{1}{\sigma^2}\text{Tr}\big(\text{Cov}(\hat y,y)\big)
$$
where the variance can be estimated by 
$$
\frac{1}{N-p-1}\sum_{i=1}^N
$$
To estimate the covariance, we will use a method called __residual bootstraping__. The idea is to estimate the covariance by creating a discrete distribution $\mathcal{R}$ of the residual errors 
$$
\mathcal{R}=\{\hat e_i = y_i-\hat y_i\}\,,
$$
and repeatedly sampling it to estimate the covariance. Here, the hat on $\hat e$ just indicates that this is dependent on the model fit $\hat f$. 

The algorithm is:

<div class="alert alert-block alert-success">
<b>Residual bootstraping</b><br>

<ul>
    <li>For $i=1,...,N$ create a new set of bootstrap samples $\hat b_{i} = \hat y_i + e$ where $e$ is drawn uniformly from the distribution of residuals $\mathcal{R}$.</li>
    <li>Recompute the fit $h^{(b)}$ on the samples $(x_i, b_i)$, and use the new sample data to compute $\text{Cov}(b_i,y_i)$</li>
    
</ul>
</div>

## Problems:

### Problem 1: Bootstrapping a Confidence Interval

If we don't have a formula for the confidence interval of a statistic, we can often estimate it by sampling from out data set many times, computing the statistic of interest, and then plotting the distribution. This is known as __bootstrapping__ the confidence interval, since you're using the data to make estimates about your fits, effectively pulling yourself up by your bootstraps. In this problem, we will see how to boot strap the confidence interval for the $\beta$ parameters in the linear fit. 

Lets return to the one variable examples of fitting the sales price to the first floor square footage __1stFlrSF__. Using a for loop, compute $\beta_0$ and $\beta_1$ 1000 times for samples of size $N = 1436$ _with replacement_ and store their results in vectors, as in the code below. 


In [ ]:
N = 1000

beta0 = np.zeros(N)
beta1 = np.zeros(N)

for i in range(N):
    ## Compute beta0 and beta1, using linear algebra, sklearn, or scipy
    beta0[i] = 
    beta1[i] = 
    

__Turn in__

1. Plot a histogram of $\beta_0$ and $\beta_1$. 
2. Using `beta0.sort()`, sort the values and find the interval containing the middle 950 values. This is the bootstrap 95% confidence interval. 
3. Using the formulas from Lecture 3, compute the confidence interval using the $z$-score. Remember that here you use all of the training data. Compare your results. 

### Problem 2: Linear Methods on High Dimensional Data

Perform ridge regression and lasso regression on the MRI Slices dataset on blackboard. You should follow the __Loading the Viewing MRI Slices__ notebook, eventually loading all slices into Python as a data matrix, with all picture dimensions flattened. The text and code for that process has been reproduced below.

We want to fit the MRI Slices data to the label __CDR__ in the labels data.


__Turn in__: 

1. Given the train-test split with seed $255$, what is the best $\alpha$ value for pure Ridge Regression? Justify your answer. 
2. Given the train-test split with seed $255$, what is the best $\lambda$ value for pure Lasso Regression? Justify your answer. 
3. (Bonus) What is the best $(\alpha,\lambda)$ value for elastic net regression?

You may set the downsample rate to higher you are unable to compute the linear model.

### Load MRI All Files

To load all of the files into an array we need to be able to search through the directory. Luckily, this is easy to do using the labels file, since each file name is stored there. We just need to loop through the __Filename__ column in the `labels` dataset and load them into an array one by one. There are 702 files in total. 

With the array there are two ways we can load them in: First, we can load them into a $702\times 176 \times 208$ array, which is the best option if we care about the 2D structure. However for algorithms like linear regression that done see the 2D structure, we may want to flatten the images to a $702\times 36608$ array (note that $36608 = 176 \times 208$). Its easy enough two switch back and forth between the two array structures later. We will start with the flattened array. 

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib

file_dir = 'C:/PATH_TO_IMAGE_DIRECTORY/'

labels = pd.read_csv(file_dir + 'labels.csv')
display(labels)

In [ ]:
DS = 8             # Downsample rate, must be a multiple of 36608

if 36608/DS % 1 > 0:
    print("Downsample rate is not a multiple of 36608")
    DS = 1
    im_size = 36608
else:
    im_size = int(36608/DS)


data = np.zeros([702, im_size])

for i, file_name in enumerate(labels.Filename):
    img = np.mean(matplotlib.image.imread(file_dir + file_name),axis=2).reshape(-1)
    data[i,:] = img[::DS]            # Downsample the image